<a href="https://colab.research.google.com/github/nileshmalode11/mcq_generation-_systeam/blob/main/MCQS_question_generation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **# Installing Necessary Packages -**

In [ ]:
#Installed Packages-

!pip install gensim
!pip install git+https://github.com/boudinfl/pke.git
!python -m spacy download en
!pip install bert-extractive-summarizer --upgrade --force-reinstall
!pip install spacy==2.1.3 --upgrade --force-reinstall
!pip install -U nltk
!pip install -U pywsd
!pip install flashtext
! pip install pywsd


'\n!pip install gensim\n!pip install git+https://github.com/boudinfl/pke.git\n!python -m spacy download en\n!pip install bert-extractive-summarizer --upgrade --force-reinstall\n!pip install spacy==2.1.3 --upgrade --force-reinstall\n!pip install -U nltk\n!pip install -U pywsd\n!pip install flashtext\n! pip install pywsd'

In [ ]:
# Importing nltk-

import nltk
nltk.download('stopwords')
nltk.download('popular')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

"import nltk\nnltk.download('stopwords')\nnltk.download('popular')\nnltk.download('punkt')\nnltk.download('wordnet')\nnltk.download('averaged_perceptron_tagger')"

In [ ]:
#Importing required libraries

import pprint
import itertools
import re
import pke
import string
import requests
import random
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn



Warming up PyWSD (takes ~10 secs)... took 4.881949424743652 secs.


In [ ]:
#Input Text

text = """Gravity (from Latin gravitas, meaning 'weight'), or gravitation, is a natural phenomenon by which all
          things with mass or energy—including planets, stars, galaxies, and even light—are brought toward (or gravitate toward)
          one another. On Earth, gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides.
          The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing
          and forming stars and caused the stars to group together into galaxies, so gravity is responsible for many of
          the large-scale structures in the Universe. Gravity has an infinite range, although its effects become increasingly
          weaker as objects get further away.
          """

In [ ]:
# Function to extract nouns using the MultipartiteRank algorithm

def get_nouns_multipartite(text):
    out = []
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text)
    pos = {'PROPN'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')

    extractor.candidate_selection(pos=pos)

    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
    keyphrases = extractor.get_n_best(n=20)
    for key in keyphrases:
        out.append(key[0])
    return out

keywords = get_nouns_multipartite(text)
print(keywords)
filtered_keys = []
for keyword in keywords:
    if keyword.lower() in text.lower():
        filtered_keys.append(keyword)

print(filtered_keys)

['moon', 'earth', 'universe', 'gravity']
['moon', 'earth', 'universe', 'gravity']


In [ ]:
# Function to tokenize text into sentences

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

In [ ]:
# Function to map keywords to sentences containing them

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)
    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences
sentences = tokenize_sentences(text)
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)

print (keyword_sentence_mapping)

{'moon': ["On Earth, gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides."], 'earth': ["On Earth, gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides."], 'universe': ['The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing \n          and forming stars and caused the stars to group together into galaxies, so gravity is responsible for many of \n          the large-scale structures in the Universe.', 'The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing \n          and forming stars and caused the stars to group together into galaxies, so gravity is responsible for many of \n          the large-scale structures in the Universe.'], 'gravity': ['The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing \n          and forming stars and caused th

In [ ]:
# Function to retrieve distractors from WordNet based on a given synset and word

def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0:
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

In [ ]:
# Function to get the most appropriate word sense for a given word in a sentence
def get_wordsense(sent,word):
    word= word.lower()

    if len(word.split())>0:
        word = word.replace(" ","_")


    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

In [ ]:
# Function to retrieve distractors from ConceptNet
# Distractors from http://conceptnet.io/

def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = []
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term']

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)

    return distractor_list

key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:

        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

index = 1
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    print ("%s)"%(index),output)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1

1) On Earth, gravity gives weight to physical objects, and the  _______ 's gravity causes the ocean tides.
	 a )   Catch
	 b )   Commemorative
	 c )   Charm
	 d )   Moon

More options:  ['Curio', 'Discard', 'Draw', 'Film', 'Finding', 'Floater', 'Fomite', 'Geological Formation', 'Growth', 'Hail', 'Head', 'Hoodoo', 'Ice', 'Je Ne Sais Quoi', 'Keepsake', 'Land'] 


2) On  _______ , gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides.
	 a )   Commemorative
	 b )   Catch
	 c )   Earth
	 d )   Charm

More options:  ['Curio', 'Discard', 'Draw', 'Film', 'Finding', 'Floater', 'Fomite', 'Geological Formation', 'Growth', 'Hail', 'Head', 'Hoodoo', 'Ice', 'Je Ne Sais Quoi', 'Keepsake', 'Land'] 


3) The gravitational attraction of the original gaseous matter present in the  _______  caused it to begin coalescing 
          and forming stars and caused the stars to group together into galaxies, so gravity is responsible for many of 
          the large-scale struct

# 👉**Another Approches -**

## **We can use Pre-trained model for this task like -**

1] BERT

2] T5 - Text-to-text transfer transformer

3] GPT - Generative Pre-trained Transformer

4] Transformer-XL

5] Rule-Based Systems with NLP Techniques